# Generating Timeseries Synthetic Data using PAR model

From SDV online manual

In [9]:
from sdv.demo import load_timeseries_demo
from sdv.timeseries import PAR

In [10]:
data = load_timeseries_demo()
data.head()

,Symbol,Date,Open,Close,Volume,MarketCap,Sector,Industry
0,AAPL,2018-12-31,39.632500,39.435001,140014000,7.378734e+11,Technology,Computer Manufacturing
1,AAPL,2019-01-02,38.722500,39.480000,148158800,7.378734e+11,Technology,Computer Manufacturing
2,AAPL,2019-01-03,35.994999,35.547501,365248800,7.378734e+11,Technology,Computer Manufacturing
3,AAPL,2019-01-04,36.132500,37.064999,234428400,7.378734e+11,Technology,Computer Manufacturing
4,AAPL,2019-01-07,37.174999,36.982498,219111200,7.378734e+11,Technology,Computer Manufacturing


In [11]:
data.shape

(25784, 8)

In [21]:
# learn model
# entity: the column that says, this is a single entity
entity_columns = ['Symbol']
# context: the columns that say, for a single entity, these should be fixed
context_columns = ['MarketCap', 'Sector', 'Industry']
# sequence index: the column that denotes the timeseries-ness
sequence_index = 'Date'
# field_types: to tell the format of output expected for a column
field_types = {
    'Symbol': {
        'type': 'id',
        'subtype' : 'string',
        'regex': '[A-Z]{3,4}'
    }
}

model = PAR(
    entity_columns = entity_columns,
    context_columns = context_columns,
    sequence_index = sequence_index,
    field_types = field_types,
)
model.fit(data)

/Users/mamduhzabidi/opt/anaconda3/envs/ML_2022/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/mamduhzabidi/opt/anaconda3/envs/ML_2022/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/mamduhzabidi/opt/anaconda3/envs/ML_2022/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Users/mamduhzabidi/opt/anaconda3/envs/ML_2022/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Users/mamduhzabidi/opt/anaconda3/envs/ML_2022/lib/python3.7/site-packages/scipy/optimize/minpack.py:

In [22]:
# save model
model.save('PARtestmodel.pkl')

In [23]:
# reload model
l_model = PAR.load('PARtestmodel.pkl')

In [24]:
# generate synthetitic data using the loaded model
l_model.sample(num_sequences=1).head()

,Symbol,Date,Open,Close,Volume,MarketCap,Sector,Industry
0,AAA,2019-01-02,334.415741,132.212193,6286675,3.055243e+10,Transportation,Railroads
1,AAA,2019-01-02,142.144138,215.666462,4724213,3.055243e+10,Transportation,Railroads
2,AAA,2019-01-04,169.480163,164.861392,1032529,3.055243e+10,Transportation,Railroads
3,AAA,2019-01-03,163.797658,163.658715,748751,3.055243e+10,Transportation,Railroads
4,AAA,2019-01-07,171.288554,163.591178,-1110256,3.055243e+10,Transportation,Railroads


In [25]:
# fix the generated synthetic data
import pandas as pd
context = pd.DataFrame([
{
    'Symbol': 'FRONTKEN',
    'MarketCap': 4e6,
    'Sector': 'Technology',
    'Industry': 'Electronic Components'
},
{
    'Symbol': 'HARTALEGA',
    'MarketCap': 12e6,
    'Sector': 'Health Care',
    'Industry': 'Medical/Nursing Services'
},
])
context

,Symbol,MarketCap,Sector,Industry
0,FRONTKEN,4000000.0,Technology,Electronic Components
1,HARTALEGA,12000000.0,Health Care,Medical/Nursing Services


In [26]:
# use the context for sampling
new_data = model.sample(context=context)
new_data

,Symbol,Date,Open,Close,Volume,MarketCap,Sector,Industry
0,FRONTKEN,2018-12-27,183.437196,141.186077,6569659,4000000.0,Technology,Electronic Components
1,FRONTKEN,2019-01-01,90.557131,183.531971,-1396360,4000000.0,Technology,Electronic Components
2,FRONTKEN,2019-01-03,143.380084,121.932152,3307965,4000000.0,Technology,Electronic Components
3,FRONTKEN,2019-01-05,131.466790,142.685023,3552392,4000000.0,Technology,Electronic Components
4,FRONTKEN,2019-01-06,130.447594,145.394549,-1656804,4000000.0,Technology,Electronic Components
...,...,...,...,...,...,...,...,...
371,HARTALEGA,2019-08-24,134.509558,139.465339,2125899,12000000.0,Health Care,Medical/Nursing Services
372,HARTALEGA,2019-08-22,183.437196,127.027417,2253717,12000000.0,Health Care,Medical/Nursing Services
373,HARTALEGA,2019-08-25,132.585359,137.863956,4764140,12000000.0,Health Care,Medical/Nursing Services
374,HARTALEGA,2019-08-27,122.352240,170.700819,3497008,12000000.0,Health Care,Medical/Nursing Services
